In [1]:
import numpy as np 
import pandas as pd


In [2]:
df = pd.read_csv("/Users/andreferdinand/Desktop/MOPT/MK/gym-MK/gym_MK/envs/Products.csv")

In [3]:
df

,PRODUCT_ID,PRODUCT,STORAGE,ENDPRODUCT,MIN,PROCESS_00,PROCESS_01,DEMAND_DIST,DIST_PARA,PROCESS_02
0,0,DOUGH,30,False,100,BREAD,BUNS,NaN,NaN,NaN
1,1,BREAD,10,True,70,NaN,NaN,NORMAL,"(1, 0.1)",NaN
2,2,BUNS,10,False,70,BUNS1,BUNS2,NaN,NaN,BUNS3
3,3,BUNS1,10,True,10,NaN,NaN,NORMAL,"(1, 0.1)",NaN
4,4,BUNS2,10,True,10,NaN,NaN,NORMAL,"(1, 0.1)",NaN
5,5,BUNS3,10,True,10,NaN,NaN,NORMAL,"(1, 0.1)",NaN


In [4]:
from helper_stats import min_prod, state
min_product = min_prod()
game_state =state()

In [5]:
game_state

In [6]:
from MK_env2 import MKEnv

In [7]:
game = MKEnv()

In [8]:
game.observation_assignment_rev

{'STORAGE_DOUGH': 0,
 'STORAGE_BREAD': 1,
 'STORAGE_BUNS': 2,
 'STORAGE_BUNS1': 3,
 'STORAGE_BUNS2': 4,
 'STORAGE_BUNS3': 5,
 'DEMAND_BREAD': 6,
 'DEMAND_BUNS1': 7,
 'DEMAND_BUNS2': 8,
 'DEMAND_BUNS3': 9}

In [9]:
game.action_assignment_rev

{'BREAD': 0, 'BUNS1': 1, 'BUNS2': 2, 'BUNS3': 3}

In [10]:
game.game_state 

(30, 10, 10, 10, 10, 10, 0, 0, 0, 0)

In [11]:
game._get_prod((10,10,10,10))

(50, 20, 60, 20, 20, 20, 0, 0, 0, 0)

In [19]:
game.observation_assignment_rev

{'STORAGE_DOUGH': 0,
 'STORAGE_BREAD': 1,
 'STORAGE_BUNS': 2,
 'STORAGE_BUNS1': 3,
 'STORAGE_BUNS2': 4,
 'STORAGE_BUNS3': 5,
 'DEMAND_BREAD': 6,
 'DEMAND_BUNS1': 7,
 'DEMAND_BUNS2': 8,
 'DEMAND_BUNS3': 9}

In [18]:
df

,PRODUCT_ID,PRODUCT,STORAGE,ENDPRODUCT,MIN,PROCESS_00,PROCESS_01,DEMAND_DIST,DIST_PARA,PROCESS_02
0,0,DOUGH,30,False,100,BREAD,BUNS,NaN,NaN,NaN
1,1,BREAD,10,True,70,NaN,NaN,NORMAL,"(1, 0.1)",NaN
2,2,BUNS,10,False,70,BUNS1,BUNS2,NaN,NaN,BUNS3
3,3,BUNS1,10,True,10,NaN,NaN,NORMAL,"(1, 0.1)",NaN
4,4,BUNS2,10,True,10,NaN,NaN,NORMAL,"(1, 0.1)",NaN
5,5,BUNS3,10,True,10,NaN,NaN,NORMAL,"(1, 0.1)",NaN


In [46]:
start_space = dict(["STORAGE_"+i, int(df["STORAGE"][df["PRODUCT"]==i])] for i in df["PRODUCT"])
liste.update(dict(["DEMAND_"+j, 0] for j in df["PRODUCT"][df["ENDPRODUCT"]]))
liste

{'STORAGE_DOUGH': 10,
 'STORAGE_BREAD': 10,
 'STORAGE_BUNS': 10,
 'STORAGE_BUNS1': 10,
 'STORAGE_BUNS2': 10,
 'STORAGE_BUNS3': 10,
 'DEMAND_BREAD': 0,
 'DEMAND_BUNS1': 0,
 'DEMAND_BUNS2': 0,
 'DEMAND_BUNS3': 0}

In [59]:
dependencies_dict = dict()

In [77]:
dependencies_dict = dict()
gen = (j for j in df.columns if j.startswith("PROCESS"))
for j in gen:
    print(j)
    

PROCESS_00
PROCESS_01
PROCESS_02


In [161]:
dependencies_dict = dict()
gen = (j for j in df.columns if j.startswith("PROCESS"))
for j in gen:
    dependencies_dict.update(pd.Series(df[df[j].notnull()].PRODUCT.values,index=getattr(df[df[j].notnull()],j)).to_dict())
    

In [162]:
dependencies_dict

{'BREAD': 'DOUGH',
 'BUNS1': 'BUNS',
 'BUNS': 'DOUGH',
 'BUNS2': 'BUNS',
 'BUNS3': 'BUNS'}

In [211]:
import copy
dependencies_dict_copy = copy.deepcopy(dependencies_dict)
action = (10,10,10,10)
liste = list(game.game_state)
action_dict = dict([[j,0] for j in df["PRODUCT"][~df["ENDPRODUCT"]]])
for i in game.endproducts:
            liste[game.observation_assignment_rev["STORAGE_"+i]] += action[game.action_assignment_rev[i]]
            action_dict[dependencies_dict[i]] += action[game.action_assignment_rev[i]]
            dependencies_dict_copy.pop(i)

In [212]:
action_dict

{'DOUGH': 10, 'BUNS': 30}

In [213]:
notfin = [j for j in df["PRODUCT"][~df["ENDPRODUCT"]]]
print(notfin)
while notfin:
    print(notfin)
    while list(set(notfin)-set(list(dependencies_dict_copy.values()))):
        print(dependencies_dict_copy)
        for j in list(set(notfin)-set(list(dependencies_dict_copy.values()))):
            print(action_dict[j])
            print(liste[game.observation_assignment_rev["STORAGE_"+j]])
            print(j)
            if liste[game.observation_assignment_rev["STORAGE_"+j]]>=action_dict[j]:
                liste[game.observation_assignment_rev["STORAGE_"+j]] -= action_dict[j]
            else:
                try:
                    action_dict[dependencies_dict_copy[j]] += max(action_dict[j]-liste[game.observation_assignment_rev["STORAGE_"+j]], getattr(game.min_product,"MIN_"+j))
                    liste[game.observation_assignment_rev["STORAGE_"+j]] += -action_dict[j] + action_dict[dependencies_dict_copy[j]]
                except:
                    liste[game.observation_assignment_rev["STORAGE_"+j]] += -action_dict[j] + max(action_dict[j]-liste[game.observation_assignment_rev["STORAGE_"+j]],getattr(game.min_product,"MIN_"+j))  
            notfin.remove(j)
            try: 
                dependencies_dict_copy.pop(j)
            except:
                pass

['DOUGH', 'BUNS']
['DOUGH', 'BUNS']
{'BUNS': 'DOUGH'}
30
10
BUNS
['DOUGH']
{}
{}
80
30
DOUGH
[]
{}


In [215]:
print(liste)
action

[50, 20, 60, 20, 20, 20, 0, 0, 0, 0]


(10, 10, 10, 10)

In [176]:
list(dict_new.values())

['DOUGH']

In [177]:
notfin

['DOUGH', 'BUNS']

In [178]:
list(set(notfin)-set(list(dict_new.values())))

['BUNS']